# Imports

In [ ]:
from PIL import Image
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset
import albumentations as A
import zipfile
import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Unet/')

In [ ]:
class DataGen(keras.utils.Sequence):
    def __init__(self, images,masks, batch_size=8, image_size=256):
        self.images = images
        self.masks = masks
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()

    def __load__(self, index):
        image = self.images[index]
        image = cv2.resize(image, (self.image_size, self.image_size))

        mask = self.masks[index]
        mask = cv2.resize(mask,(self.image_size,self.image_size))

        ## Normalizaing
        image = image/255.0
        mask = mask

        return image, mask

    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.images):
            self.batch_size = len(self.images) - index*self.batch_size

        indexes = range(index*self.batch_size,(index+1)*self.batch_size)

        image = []
        mask  = []

        for i in indexes:
            _img, _mask = self.__load__(i)
            image.append(_img)
            mask.append(_mask)

        image = np.array(image)
        mask  = np.array(mask)

        return image, mask

    def on_epoch_end(self):
        pass

    def __len__(self):
        return int(np.ceil(len(self.images)/float(self.batch_size)))

In [ ]:
image = ['/content/aerial.jpg','/content/aerial2.jpg','/content/aerial3.jpg']
images = []
for path in image:
  img = Image.open(path)
  img = np.array(img)
  images.append(img)
image_gen = DataGen(images,images,1)

In [ ]:
x,y = image_gen.__getitem__(2)

In [ ]:
out = model.predict(x)

1/1 [==============================] - 1s 675ms/step
